In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

In [9]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

KeyboardInterrupt: 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(4, 4)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
X = np.random.uniform(0, 1, size=(300, 1))
# y = X[:, 0] - X[:, 1]**2 + 2 * X[:, 2] * X[:, 3] + (1 / ((1 + X[:, 4]) ** 6))
y = np.sinc(X).sum(axis=1)

torch_data = TensorDataset(torch.Tensor(X), torch.Tensor(y))
data_loader = DataLoader(torch_data, batch_size=100)

In [ ]:
model = Net()
# loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, model=model, use_diagonal=False)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(trainloader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x)
        opt.update(loss)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] < all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().numpy().item()))

    if patience <= 0:
        break

epoch:  0

KeyboardInterrupt: 

In [ ]:
model = Net()
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=0.01, ld=1, model=model, use_diagonal=True)

all_loss = {}
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x)
        opt.update(loss)

        all_loss[epoch + 1] += loss

    all_loss[epoch + 1] /= len(data_loader)

    if epoch > 0 and all_loss[epoch] < all_loss[epoch + 1]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.8393998146057129
epoch:  1, loss: 0.8350479006767273
epoch:  2, loss: 125.46124267578125
epoch:  3, loss: 756704.3125
epoch:  4, loss: 203142447104.0
epoch:  5, loss: 534557229056.0
epoch:  6, loss: 534760751104.0
epoch:  7, loss: 534822748160.0
epoch:  8, loss: 534839033856.0
epoch:  9, loss: 534839296000.0
epoch:  10, loss: 534839427072.0
epoch:  11, loss: 534839427072.0
epoch:  12, loss: 534839427072.0
epoch:  13, loss: 534839427072.0
epoch:  14, loss: 534839427072.0
epoch:  15, loss: 534839427072.0
epoch:  16
Intel MKL ERROR: Parameter 4 was incorrect on entry to SLASCL.

Intel MKL ERROR: Parameter 4 was incorrect on entry to SLASCL.


_LinAlgError: linalg.svd: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated singular values (error code: 23).